In [1]:
import torch
import esm
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz
from umap import UMAP
import plotly.express as px
# pip install tables
%matplotlib inline

In [4]:
# retrieve all proteins from the dataset
df = pd.read_csv('USE__mutant_seqs_WITHscore.csv', index_col=0)
#df = pd.read_csv('skempiwmutants_nanincl.csv', index_col=0)
df = df[['#Pdb','Mutation(s)_PDB', 'Affinity_mut_parsed','Affinity_wt_parsed','Protein 1', 'Protein 2', 'wild_seq1', 'wild_seq2', 'mutant_seq']]
df

,#Pdb,Mutation(s)_PDB,Affinity_mut_parsed,Affinity_wt_parsed,Protein 1,Protein 2,wild_seq1,wild_seq2,mutant_seq
0,1CSE_E_I,LI45G,5.260000e-11,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTGD...
1,1CSE_E_I,LI45S,8.330000e-12,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTSD...
2,1CSE_E_I,LI45P,1.020000e-07,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTPD...
3,1CSE_E_I,LI45I,1.720000e-10,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTID...
4,1CSE_E_I,LI45D,1.920000e-09,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTDD...
...,...,...,...,...,...,...,...,...,...
3411,3QIB_ABP_CD,AA64G,1.540000e-05,5.500000e-06,I-Ek plus MCC peptide,2B4 TCR,IKEEHTIIQAEFYLLPDKRGEFMFDFDGDEIFHVDIEKSETIWRLE...,MANERADLIAYLKQATKGGGGSGGGGGMKVIQTPRYLVKGQGQKAK...,IKEEHTIIQAEFYLLPDKRGEFMFDFDGDEIFHVDIEKSETIWRLE...
3412,3QIB_ABP_CD,KA67A,1.430000e-05,5.500000e-06,I-Ek plus MCC peptide,2B4 TCR,IKEEHTIIQAEFYLLPDKRGEFMFDFDGDEIFHVDIEKSETIWRLE...,MANERADLIAYLKQATKGGGGSGGGGGMKVIQTPRYLVKGQGQKAK...,IKEEHTIIQAEFYLLPDKRGEFMFDFDGDEIFHVDIEKSETIWRLE...
3413,3QIB_ABP_CD,AA68G,2.290000e-05,5.500000e-06,I-Ek plus MCC peptide,2B4 TCR,IKEEHTIIQAEFYLLPDKRGEFMFDFDGDEIFHVDIEKSETIWRLE...,MANERADLIAYLKQATKGGGGSGGGGGMKVIQTPRYLVKGQGQKAK...,IKEEHTIIQAEFYLLPDKRGEFMFDFDGDEIFHVDIEKSETIWRLE...
3414,3QIB_ABP_CD,EA75A,5.500000e-06,5.500000e-06,I-Ek plus MCC peptide,2B4 TCR,IKEEHTIIQAEFYLLPDKRGEFMFDFDGDEIFHVDIEKSETIWRLE...,MANERADLIAYLKQATKGGGGSGGGGGMKVIQTPRYLVKGQGQKAK...,IKEEHTIIQAEFYLLPDKRGEFMFDFDGDEIFHVDIEKSETIWRLE...


In [5]:
# if there's ":" in the column, just drop first, indicating multi-chain situation
ignore = (df['wild_seq1'].str.contains(':')) | (df['wild_seq2'].str.contains(':')) | (df['mutant_seq'].str.contains(':'))
reduced_df = df[~ignore]
reduced_df

,#Pdb,Mutation(s)_PDB,Affinity_mut_parsed,Affinity_wt_parsed,Protein 1,Protein 2,wild_seq1,wild_seq2,mutant_seq
0,1CSE_E_I,LI45G,5.260000e-11,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTGD...
1,1CSE_E_I,LI45S,8.330000e-12,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTSD...
2,1CSE_E_I,LI45P,1.020000e-07,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTPD...
3,1CSE_E_I,LI45I,1.720000e-10,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTID...
4,1CSE_E_I,LI45D,1.920000e-09,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTDD...
...,...,...,...,...,...,...,...,...,...
3403,2WPT_A_B,FB86A,7.410000e-05,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...
3404,2WPT_A_B,"DA33A,FB86A",6.210000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...
3405,2WPT_A_B,NA34A,6.490000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDAKLVREFERLTEH...
3406,2WPT_A_B,SB84A,7.090000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...


In [6]:
# check and remove if there's any duplicates
reduced_df.drop_duplicates(inplace=True)
df_ = reduced_df.copy()
# shouldn't drop na, na means lose binding ability in mutants
#df_.dropna(inplace=True)
df_['wild_seq_1'] = df_['wild_seq1']
df_['wild_seq_2'] = df_['wild_seq2']
df_['Protein_1'] = df_['Protein 1']
df_['Protein_2'] = df_['Protein 2']
df_ = df_.reset_index(drop=True)
df_

/var/tmp/pbs.823591.hn-10-03/ipykernel_171837/2461256375.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df.drop_duplicates(inplace=True)


,#Pdb,Mutation(s)_PDB,Affinity_mut_parsed,Affinity_wt_parsed,Protein 1,Protein 2,wild_seq1,wild_seq2,mutant_seq,wild_seq_1,wild_seq_2,Protein_1,Protein_2
0,1CSE_E_I,LI45G,5.260000e-11,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTGD...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
1,1CSE_E_I,LI45S,8.330000e-12,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTSD...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
2,1CSE_E_I,LI45P,1.020000e-07,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTPD...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
3,1CSE_E_I,LI45I,1.720000e-10,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTID...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
4,1CSE_E_I,LI45D,1.920000e-09,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTDD...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,2WPT_A_B,FB86A,7.410000e-05,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,Colicin E2 immunity protein,Colicin E9 DNase
2022,2WPT_A_B,"DA33A,FB86A",6.210000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,Colicin E2 immunity protein,Colicin E9 DNase
2023,2WPT_A_B,NA34A,6.490000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDAKLVREFERLTEH...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,Colicin E2 immunity protein,Colicin E9 DNase
2024,2WPT_A_B,SB84A,7.090000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,Colicin E2 immunity protein,Colicin E9 DNase


In [7]:
"""
    This chunk should re-organize the dataframe so that:
    1. all the sequence listed in wt_seq2 will have positions mutated
    2. if the wt_seq2 and wt_seq1 flipped, so well columns [Protein1, Protein2]
"""
for index, row in df_.iterrows():
    if fuzz.ratio(row['wild_seq1'], row['mutant_seq']) > fuzz.ratio(row['wild_seq2'], row['mutant_seq']) :
        # then the two sequences are similar
        # flip the sequence 1 to sequence 2
            df_.at[index, 'wild_seq_1'] = row['wild_seq2']
            df_.at[index, 'wild_seq_2'] = row['wild_seq1']
            df_.at[index, 'Protein_1'] = row['Protein 2']
            df_.at[index, 'Protein_2'] = row['Protein 1']
    elif fuzz.ratio(row['wild_seq1'], row['mutant_seq'])  < fuzz.ratio(row['wild_seq2'], row['mutant_seq']) :
        pass
    else:
        print(index, fuzz.ratio(row['wild_seq1'], row['mutant_seq']), fuzz.ratio(row['wild_seq2'], row['mutant_seq']))
        print('mutate both sequences?')
df_

,#Pdb,Mutation(s)_PDB,Affinity_mut_parsed,Affinity_wt_parsed,Protein 1,Protein 2,wild_seq1,wild_seq2,mutant_seq,wild_seq_1,wild_seq_2,Protein_1,Protein_2
0,1CSE_E_I,LI45G,5.260000e-11,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTGD...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
1,1CSE_E_I,LI45S,8.330000e-12,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTSD...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
2,1CSE_E_I,LI45P,1.020000e-07,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTPD...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
3,1CSE_E_I,LI45I,1.720000e-10,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTID...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
4,1CSE_E_I,LI45D,1.920000e-09,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTDD...,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,Subtilisin Carlsberg,Eglin c
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,2WPT_A_B,FB86A,7.410000e-05,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,Colicin E2 immunity protein,Colicin E9 DNase
2022,2WPT_A_B,"DA33A,FB86A",6.210000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,Colicin E2 immunity protein,Colicin E9 DNase
2023,2WPT_A_B,NA34A,6.490000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDAKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,Colicin E9 DNase,Colicin E2 immunity protein
2024,2WPT_A_B,SB84A,7.090000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,Colicin E2 immunity protein,Colicin E9 DNase


In [8]:
df_.drop(columns=['Protein 1', 'Protein 2', 'wild_seq1', 'wild_seq2'], inplace=True)
cols = ['#Pdb', 'Mutation(s)_PDB', 'Affinity_mut_parsed', 'Affinity_wt_parsed','Protein_1',
        'Protein_2','wild_seq_1','wild_seq_2', 'mutant_seq']
df_ = df_[cols]
df_

,#Pdb,Mutation(s)_PDB,Affinity_mut_parsed,Affinity_wt_parsed,Protein_1,Protein_2,wild_seq_1,wild_seq_2,mutant_seq
0,1CSE_E_I,LI45G,5.260000e-11,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTGD...
1,1CSE_E_I,LI45S,8.330000e-12,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTSD...
2,1CSE_E_I,LI45P,1.020000e-07,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTPD...
3,1CSE_E_I,LI45I,1.720000e-10,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTID...
4,1CSE_E_I,LI45D,1.920000e-09,1.120000e-12,Subtilisin Carlsberg,Eglin c,AQTVPYGIPLIKADKVQAQGFKGANVKVAVLDTGIQASHPDLNVVG...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTLD...,TEFGSELKSFPEVVGKTVDQAREYFTLHYPQYNVYFLPEGSPVTDD...
...,...,...,...,...,...,...,...,...,...
2021,2WPT_A_B,FB86A,7.410000e-05,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...
2022,2WPT_A_B,"DA33A,FB86A",6.210000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...
2023,2WPT_A_B,NA34A,6.490000e-07,8.200000e-07,Colicin E9 DNase,Colicin E2 immunity protein,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MELKHSISDYTEAEFLEFVKKIARAEGATECDDAKLVREFERLTEH...
2024,2WPT_A_B,SB84A,7.090000e-07,8.200000e-07,Colicin E2 immunity protein,Colicin E9 DNase,MELKHSISDYTEAEFLEFVKKIARAEGATECDDNKLVREFERLTEH...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...,MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKE...


In [17]:
with open('SKEMPI_wild_seq_1.fasta', 'w') as f:
    for seq, group in df_.drop_duplicates('wild_seq_1').groupby('wild_seq_1'):
        for _, row in group.iterrows():
            header = f">{row['#Pdb']}|{row['Protein_1']}"
            f.write(f"{header}\n{seq}\n")

# Write unique sequences from wild_seq_2 to a FASTA file
with open('SKEMPI_wild_seq_2.fasta', 'w') as f:
    for seq, group in df_.drop_duplicates('wild_seq_2').groupby('wild_seq_2'):
        for _, row in group.iterrows():
            header = f">{row['#Pdb']}|{row['Protein_2']}"
            f.write(f"{header}\n{seq}\n")

# Write unique sequences from mutant_seq to a FASTA file
with open('SKEMPI_mutant_seq.fasta', 'w') as f:
    for seq, group in df_.drop_duplicates('mutant_seq').groupby('mutant_seq'):
        for _, row in group.iterrows():
            header = f">{row['#Pdb']}|{row['Mutation(s)_PDB']}|{row['Protein_2']}"
            f.write(f"{header}\n{seq}\n")

In [9]:
# Generate sequence embeddings to the proteins
seqs_wt1 = df_.wild_seq_1.values.tolist()
seqs_wt2 = df_.wild_seq_2.values.tolist()
seqs_mut = df_.mutant_seq.values.tolist()
seqs_wt1 = set(seqs_wt1)
seqs_wt2 = set(seqs_wt2)
seqs_mut = set(seqs_mut)
seqs_mut

{'MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKEFKSFDDFRKAVWEEVSKDPELSKNLNPSNKSSVSKGYSPATPKNQQVGCRKVYELHHDKPISQGGEVYDMDNIRVTTPKRHIDIHRGK',
 'MEGWQRAFVLHSRPWSETSLMLDVFTEESGRVRLVAKGARSKRSTLKGALQPFTPLLLRFGGRGEVKTLRSAEAVSLALPLSGITLYSGLYINELLSRVLEYETRFSELFFDYLHCIQSLAGVTGTPEPALRRFELALLGHLGYGVNFTHCAGSGEPVDDTMTYRYREEKGFIASVVIDNKTFTGRQLKALNAREFPDADTLRAAKRFTRMALKPYLGGKPLKSRELDRQFMPKRTVKTHYE',
 'RPDFCLEPPYTAPCKARIIRYFYNAKAGLCQTFVYGGCRAKRNNFKSAEDCMRTCGGA',
 'MESKRNKPGKATGKGKPVGDKWLDDAGKDSGAPIPDRIADKLRDKEFKSFDDFRKAVWEEVSKDPELSKNLNPSNKSSVSKGYSPFTPKNQQVGGRAVYELHHDKPISQGGEVYDMDNIRVTTPKRHIDIHRGK',
 'MREYKLVVLGSGGVGKSALTVQFVQGIFVDEYDPTIADSYRKQVEVDCQQCMLEILDTAGTEQFTAMRDLYMKNGQGFALVYSITAQSTFNDLQDLREQILRVKDTEDVPMILVGNKCDLEDERVVGKEQGQNLARQWCNCAFLESSAKSKINVNEIFYDLVRQINR',
 'FPTIPLSRLFDNAMLRAHRLHQLAFDTYQEFEEAYIPKEQKYSFLQNPQTSLCFSESIPTPSNREETQQKSNLELLRISLLLIQSWLEPVQFLRSVFANSLVYGASDSNVYDLLKDLEERIQTLMGRLEDGSPRTGQIFKQTYSKFDTNSHNDDALLKNYGLLYCFRKDMDKVATFLRIVQCRSVEGSCGF',
 'ADPPPVHDTDGHELRADANYYVLSANRAHGGGLTMAPGHGRH

In [10]:
"""lazy to write function, may need to modify in the future"""
seqs_labeled_wt1 = []
count = 0
for seq in seqs_wt1:
    seqs_labeled_wt1.append(tuple((str('seq' + str(count)), seq)))
    count += 1
seqs_labeled_wt2 = []
count = 0
for seq in seqs_wt2:
    seqs_labeled_wt2.append(tuple((str('seq' + str(count)), seq)))
    count += 1
seqs_labeled_mut = []
count = 0
for seq in seqs_mut:
    seqs_labeled_mut.append(tuple((str('seq' + str(count)), seq)))
    count += 1

In [11]:
# Load ESM-2 model
model, alphabet = esm.pretrained.esm2_t33_650M_UR50D()
batch_converter = alphabet.get_batch_converter()
model.eval()

ESM2(
  (embed_tokens): Embedding(33, 1280, padding_idx=1)
  (layers): ModuleList(
    (0): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (q_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (out_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (rot_emb): RotaryEmbedding()
      )
      (self_attn_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=1280, out_features=5120, bias=True)
      (fc2): Linear(in_features=5120, out_features=1280, bias=True)
      (final_layer_norm): LayerNorm((1280,), eps=1e-05, elementwise_affine=True)
    )
    (1): TransformerLayer(
      (self_attn): MultiheadAttention(
        (k_proj): Linear(in_features=1280, out_features=1280, bias=True)
        (v_proj): Linear(in_features=1280, out_features=1280, bia

In [12]:
# alternative way to generate batches
from torch.utils.data import TensorDataset
from esm import Alphabet, FastaBatchedDataset
batch_size = 2000
dataset = FastaBatchedDataset(list(zip(*seqs_labeled_wt1))[0], list(zip(*seqs_labeled_wt1))[1])
batches = dataset.get_batch_indices(batch_size, extra_toks_per_seq=1)
data_loader = torch.utils.data.DataLoader(dataset, collate_fn=Alphabet.from_architecture("roberta_large").get_batch_converter(),
            batch_sampler=batches, pin_memory=True)
dataset_seq2 = FastaBatchedDataset(list(zip(*seqs_labeled_wt2))[0], list(zip(*seqs_labeled_wt2))[1])
batches_seq2 = dataset_seq2.get_batch_indices(batch_size, extra_toks_per_seq=1)
data_loader_seq2 = torch.utils.data.DataLoader(dataset_seq2, collate_fn=Alphabet.from_architecture("roberta_large").get_batch_converter(),
            batch_sampler=batches_seq2, pin_memory=True)
dataset_mut = FastaBatchedDataset(list(zip(*seqs_labeled_mut))[0], list(zip(*seqs_labeled_mut))[1])
batches_mut = dataset_mut.get_batch_indices(batch_size, extra_toks_per_seq=1)
data_loader_mut = torch.utils.data.DataLoader(dataset_mut, collate_fn=Alphabet.from_architecture("roberta_large").get_batch_converter(),
            batch_sampler=batches_mut, pin_memory=True)

In [13]:
len(max(list(zip(*seqs_labeled_wt1))[1], key=len))

968

In [14]:
len(max(list(zip(*seqs_labeled_wt2))[1], key=len))

968

In [15]:
len(max(list(zip(*seqs_labeled_mut))[1], key=len))

968

In [ ]:
torch.cuda.empty_cache()
if torch.cuda.is_available():
    model = model.cuda()
    print('Transferred model to GPU')

In [ ]:
#QC
for batch_idx, (labels, strs, toks) in enumerate(data_loader):
    print(batch_idx,labels)

In [ ]:
representation_store_dict = {}
for batch_idx, (labels, strs, toks) in enumerate(data_loader):
    if torch.cuda.is_available():
        toks = toks.to(device='cuda', non_blocking=True)
    with torch.no_grad():
        results = model(toks, repr_layers = [33], return_contacts = True)['representations'][33]
        #results = model(toks, repr_layers = [33], return_contacts = True)['logits']
    #print(results.shape)
    results_cpu = results.to(device='cpu')
    for i, str_ in enumerate(strs):
        representation_store_dict[str_] = results_cpu[i, 1: (len(strs[i])+1)].numpy()

In [ ]:
# take the average of the representations of the proteins
# for umaps, along axis 0
sequence_embeddings = {key: np.mean(value, axis=0, keepdims=True) for key, value in representation_store_dict.items()}
#print({key: value.shape for key, value in sequence_embeddings.items()})
sequence_embeddings

In [ ]:
representation_store_dict_seq2 = {}
for batch_idx, (labels, strs, toks) in enumerate(data_loader_seq2):
    if torch.cuda.is_available():
        toks = toks.to(device='cuda', non_blocking=True)
    with torch.no_grad():
        results = model(toks, repr_layers = [33], return_contacts = True)['representations'][33]
        #results = model(toks, repr_layers = [33], return_contacts = True)['logits']
    results_cpu = results.to(device='cpu')
    for i, str_ in enumerate(strs):
        representation_store_dict_seq2[str_] = results_cpu[i, 1: (len(strs[i])+1)].numpy()
sequence_embeddings_seq2 = {key: np.mean(value, axis=0, keepdims=True) for key, value in representation_store_dict_seq2.items()}
representation_store_dict_mut = {}
for batch_idx, (labels, strs, toks) in enumerate(data_loader_mut):
    if torch.cuda.is_available():
        toks = toks.to(device='cuda', non_blocking=True)
    with torch.no_grad():
        results = model(toks, repr_layers = [33], return_contacts = True)['representations'][33]
        #results = model(toks, repr_layers = [33], return_contacts = True)['logits']
    results_cpu = results.to(device='cpu')
    for i, str_ in enumerate(strs):
        representation_store_dict_mut[str_] = results_cpu[i, 1: (len(strs[i])+1)].numpy()
sequence_embeddings_mut = {key: np.mean(value, axis=0, keepdims=True) for key, value in representation_store_dict_mut.items()}

In [ ]:
def update_embeddings(row, embedding_dict):
    """
    add embeddings to the metadata column.
    cannot do the reverse, because due to mislabel, several different protein names share the same sequences
    but as long as sequences are correct, so will the embeddings
    """
    for key, value in embedding_dict.items():
        if row == key:
            return value
df_['wild_seq_1_embeddings'] = df_['wild_seq_1'].apply(update_embeddings, embedding_dict=sequence_embeddings)
df_['wild_seq_2_embeddings'] = df_['wild_seq_2'].apply(update_embeddings, embedding_dict=sequence_embeddings_seq2)
df_['mutant_seq_embeddings'] = df_['mutant_seq'].apply(update_embeddings, embedding_dict=sequence_embeddings_mut)
df_

In [ ]:
df_.to_hdf('./outputs/dataframes/proteins_embeddings_meta.hdf', key='df', mode='w')

In [ ]:
# extract meta information to draw umaps
# need to reorganize dataframe to make the features clear for each protein sequence
ppi = df_['Protein_1'] + '---' + df_['Protein_2']
prot_1 = df_['Protein_1']
prot_2 = df_['Protein_2']
prot_mut = df_['Protein_2']+'_'+'mut'
seq_1 = df_['wild_seq_1']
seq_2 = df_['wild_seq_2']
seq_mut = df_['mutant_seq']
embed_1 = df_['wild_seq_1_embeddings']
embed_2 = df_['wild_seq_2_embeddings']
embed_mut = df_['mutant_seq_embeddings']
affinity_wt = df_['Affinity_wt_parsed']
affinity_mut = df_['Affinity_mut_parsed']
label_wt_1 = pd.Series(['wt1']*len(prot_1))
label_wt_2 = pd.Series(['wt2']*len(prot_2))
label_mut = pd.Series(['mut']*len(prot_mut))
pdbs = df_['#Pdb']
mut = df_['Mutation(s)_PDB']
mut_status_wt = pd.Series([np.nan]*len(prot_1))

In [ ]:
df_meta = pd.DataFrame({
    'PDB': pd.concat([pdbs, pdbs, pdbs], ignore_index=True),
    'Protein': pd.concat([prot_1, prot_2, prot_mut], ignore_index=True),
    'Mutation': pd.concat([mut_status_wt, mut_status_wt, mut], ignore_index=True),
    'Sequence': pd.concat([seq_1, seq_2, seq_mut], ignore_index=True),
    'PPI': pd.concat([ppi, ppi, ppi], ignore_index=True),
    'Label': pd.concat([label_wt_1, label_wt_2, label_mut], ignore_index=True),
    'Affinity': pd.concat([affinity_wt, affinity_mut, affinity_mut], ignore_index=True),
    'Embedding': pd.concat([embed_1, embed_2, embed_mut], ignore_index=True),
})
df_meta

In [ ]:
expanded_embeddings = df_meta['Embedding'].apply(lambda x: pd.Series(x[0]))
df_umap = pd.concat([df_meta, expanded_embeddings], axis=1)
df_umap.drop(['Embedding'], axis=1, inplace=True)
df_umap

In [ ]:
df_umap.drop_duplicates(inplace=True, ignore_index=True)
# weird duplicates should be dropped under the affinity column
# keep the first encounter first
# seems like a lot of wts mistakenly have affinities of mutants
#df_umap= df_umap.drop_duplicates(subset='Affinity', keep='first', ignore_index=True)
df_umap.drop_duplicates(subset=['Sequence','Affinity'], keep='first')
df_umap

In [ ]:
features = df_umap.loc[:,0:]
features

In [ ]:
umap_2d = UMAP(n_components=2, init='random', random_state=0)
proj_2d = umap_2d.fit_transform(features)
results = {'umap1': proj_2d[:, 0], 'umap2': proj_2d[:, 1]}
umap_results = pd.DataFrame(data=results)
umap_results

In [ ]:
df_umap_ = pd.concat([df_umap, umap_results], axis=1)
df_umap_

In [ ]:
color_dict = {'wt1':'#72B7B2', 'wt2':'#54A24B', 'mut':'#E45756'}
fig= px.scatter(
    df_umap_,
    x = 'umap1',
    y = 'umap2',
    color='Label',
    color_discrete_map=color_dict,
    hover_name='Protein',
    hover_data={
        'Protein': False,
        'PDB': True,
        'Label': True,
        'PPI': True,
        'Mutation':True,
        'Affinity':True,
        'umap1':False,
        'umap2':False
    }
)
fig.update_layout(template='simple_white',
                  title='SKEMPI Protein-Protein embeddings',
                  title_x=0.5
                  )
fig.show()

In [ ]:
fig.write_html("./outputs/figures/skempi_ppi_plotly.html")